# Simulation Basics

In this notebook, we'll introduce you to the basics of running a simulation of a manipulator arm in Colab. Specifically, we'll use a simulation of the [Kinova Gen3](https://www.kinovarobotics.com/en/products/gen3-robot) that we have in the lab. We've tried to make the simulation interface as close as possible to the hardware interface, so it's relatively easy to apply tools you've developed in simulation to the real robot. 

# Notebook Setup

Our simulation depends on a bunch of software (most notably [Drake](https://drake.mit.edu/)) that isn't installed by default on Google's servers. The following cell installs this software, which is pretty complex, so it will probably take a minute or two. You'll need to run a similar cell every time you use the simulator. Fortunately, once a cloud machine has been provisioned and the software intalled, it should remain availible to you for 12 hours or so. 

In [1]:
#@title Run Notebook Setup
import importlib
import sys
import os
from urllib.request import urlretrieve
import subprocess
import shutil

assert 'google.colab' in sys.modules, "This notebook is meant to be run in google colab!"

drake_url = "https://drake-packages.csail.mit.edu/tmp/drake-0.27.0-pip-bionic.tar.gz"
if importlib.util.find_spec('pydrake') is None:
    # We're in colab and don't have pydrake, so install it on the cloud machine.
    if os.path.isdir('/opt/drake'):
        shutil.rmtree('/opt/drake')
    print("Installing Drake")
    urlretrieve(drake_url, 'drake.tar.gz')
    subprocess.run(['mkdir', '/opt/drake'])
    subprocess.run(['tar', '-xzf', 'drake.tar.gz', '-C', '/opt/drake'], check=True)
    
    print("Installing other dependencies")
    subprocess.run(["pip3", "install", "meshcat"])
    subprocess.run(['apt-get', 'update', '-o', 'APT::Acquire::Retries=4', '-qq'], check=True)
    with open("/opt/drake/share/drake/setup/packages-bionic.txt", "r") as f:
        packages = f.read().splitlines()
    subprocess.run(['apt-get', 'install', '-o',
                    'APT::Acquire::Retries=4', '-o', 'Dpkg::Use-Pty=0',
                    '-qy', '--no-install-recommends'] + packages,
                    check=True)
    
    v = sys.version_info
    path = f"/opt/drake/lib/python{v.major}.{v.minor}/site-packages"
    if importlib.util.find_spec('pydrake') is None:
        sys.path.append(path)

# Start a meshcat server
print("Starting Meshcat")
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'])

# Clone our github repo
install_path = '/opt/kinova_station'
if not os.path.isdir(install_path):
    print("Cloning github repo")
    subprocess.run(['git','clone','https://github.com/vincekurtz/kinova_station.git',install_path])
sys.path.append(install_path)

print("Done!")

Installing Drake
Installing other dependencies
Starting Meshcat
Cloning github repo
Done!


You don't need to know the details of how the setup cell above works, but if you want to check them you you can just double click the cell. 

# Viewing the Simulator

One of the things that the setup code above did was define a variable called `web_url` which is a web address where you can view the simulator. The easiest way to do this is probably to open the URL in a new tab. For now it should be just a blank environment with a grid, a blue background, and nothing else. 

In [ ]:
web_url

Alternatively, you can embed the `web_url` in an html `iframe`, which allows you to view the simulator in this page, using the code below.

In [ ]:
import IPython
width=800
height=400
if web_url[:5] != 'https':
    web_url = 'https' + web_url[4:]
iframe = '<iframe src=' + web_url + ' width=' + str(width) + ' height=' + str(height) + '></iframe>'
IPython.display.HTML(iframe)

# Controlling the Robot Manually

(try to pick up the block)